# 实验项目名称：聚类分析

## 实验目的及要求：

- 掌握聚类的概念
- 掌握 k-means 算法的原理
- 掌握 DBSCAN 算法的原理
- 掌握聚类模型的构建方法
- 了解 sklearn 估计器的用法

## 实验内容：

- 读取 load_diabetes 数据集、查看数据表的基本信息
- 使用 TSNE 函数对聚类之前数据进行可视化
- 分别采用 k-means 和 DBSCAN 对数据集进行聚类模型训练
- 采用聚类评价指标对两种模型的聚类结果进行评价
- 使用 TSNE 函数对最优聚类数目的聚类结果进行可视化

模型详细参数可参考  
https://scikit-learn.org/stable/

## 实验步骤及结果：

### 1. 加载数据集，读取数据集 load_diabetes，了解数据

1. 获取数据
   Sklearn 库的 datasets 模块集成了部分数据分析的经典数据集，可以使用这些数据集进行数据预处理、建模等操作。可以通过数据集加载函数加载 load_diabetes 数据集。


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

data_row = datasets.load_diabetes()
data = data_row["data"]
target = data_row["target"]
feature = data_row["feature_names"]
df = pd.DataFrame(data, columns=feature)
df.head()

### 数据说明


'DESCR':  
Ten baseline variables, age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of 442 diabetes patients, as well as the response of interest, a quantitative measure of disease progression one year after baseline.

对 442 例糖尿病患者，分别获得了 10 个基线变量:年龄、性别、体重指数、平均血压和 6 个血清测量值，以及兴趣反应（基线后一年疾病进展的定量测量）。

**数据集特征：**

实例数：442

属性数：前 10 列是数值预测值

目标：第 11 列是基线检查后一年疾病进展的定量测量

属性信息：

- age 年龄（以年为单位）
- sex 性别
- bmi 体重指数
- bp 血压平均值
- s1 tc，T 细胞（一种白细胞）
- s2 低密度脂蛋白
- s3 高密度脂蛋白，高密度脂蛋白
- s4 tch，促甲状腺激素
- s5 ltg，拉莫三嗪
- s6 血糖，血糖水平  
  注：这 10 个特征变量中的每一个都以平均值为中心，并按标准差乘以“n_samples”（即每列的平方和总计为 1）进行缩放。


### 2.数据集可视化


In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, init="random", random_state=177, learning_rate="auto").fit(
    data
)
df_reduction = pd.DataFrame(tsne.embedding_)
df_reduction

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(x=df_reduction[0], y=df_reduction[1], c="b")

### 3.建立 K-Means 聚类模型、评价聚类模型选择最优聚类簇数目


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

silh_score = []
cali_score = []
inertia_score = []
for i in range(2, 7):
    kmeans = KMeans(n_clusters=i, random_state=123).fit(data)
    score = silhouette_score(data, kmeans.labels_)
    silh_score.append(score)
    score = calinski_harabasz_score(data, kmeans.labels_)
    cali_score.append(score)
    score = kmeans.inertia_
    inertia_score.append(score)

print(silh_score)
print(cali_score)
print(inertia_score)

plt.figure(figsize=(10, 6))
plt.plot(
    range(2, 7),
    silh_score,
    "b*--",
    linewidth=1,
    markersize=10,
    label="silhouette_score",
)
plt.legend()
plt.xlabel("cluster")
plt.ylabel("score")
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(
    range(2, 7),
    cali_score,
    "rs--",
    linewidth=1,
    markersize=10,
    label="calinski_harabasz_score",
)
plt.legend()
plt.xlabel("cluster")
plt.ylabel("score")
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(
    range(2, 7),
    inertia_score,
    "go--",
    linewidth=1,
    markersize=10,
    label="inertia_score",
)
plt.legend()
plt.xlabel("cluster")
plt.ylabel("score")
plt.show()

In [ ]:
kmeans_row = KMeans(n_clusters=2, random_state=123).fit(data)
centroids_row = kmeans_row.cluster_centers_
print("原始数据集K-Means聚类后质心为", centroids_row, "\n")

kmeans = KMeans(n_clusters=2, random_state=123).fit(df_reduction)
centroids = kmeans.cluster_centers_
print("TSNE降维,K-Means聚类后质心为", centroids)

df_reduction["label"] = kmeans.labels_
cluster_0 = df_reduction[df_reduction["label"] == 0]
cluster_1 = df_reduction[df_reduction["label"] == 1]

plt.figure(figsize=(10, 8))
plt.scatter(x=cluster_0[0], y=cluster_0[1], c="b", label="cluster_0")
plt.scatter(x=cluster_1[0], y=cluster_1[1], c="m", label="cluster_1")
plt.scatter(
    centroids[:, 0], centroids[:, 1], marker="x", color="red", s=50, label="centroids"
)

plt.legend()
plt.show()
print(df_reduction.shape)

### 4.建立 DBSCAN 聚类模型、评价聚类模型选择最优聚类簇数目


In [ ]:
from sklearn.cluster import DBSCAN

for i in np.arange(0.09, 0.11, 0.001):
    for j in np.arange(3, 10):
        DB = DBSCAN(eps=i, min_samples=j).fit(data)
        labels = DB.labels_
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)
        if n_clusters_ == 2 and n_noise_ <= 30:
            print("参数eps={},min_samples={}".format(i, j))
            print("聚类之后的簇数目有{}个".format(n_clusters_))
            print("噪声数据有{}个".format(n_noise_))
            silh_score = silhouette_score(data, DB.labels_)
            cali_score = calinski_harabasz_score(data, DB.labels_)
            print("silhouette方法得分为", silh_score)
            print("calinski_harabasz方法得分为", cali_score)
            print("---------------------------------------")

### 5. 对最优聚类数目的聚类结果进行可视化


In [ ]:
DB = DBSCAN(eps=0.101, min_samples=4).fit(data)
labels = DB.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print("聚类之后的簇数目有{}个".format(n_clusters_))
print("噪声数据有{}个".format(n_noise_))
plt.figure(figsize=(10, 8))
plt.scatter(
    x=pd.DataFrame(tsne.embedding_)[:][0],
    y=pd.DataFrame(tsne.embedding_)[:][1],
    c=DB.labels_,
)

### 6.K-Means 划分为 2 个簇后每个点的簇分类情况


In [ ]:
kmeans_label_Series = pd.Series(kmeans_row.labels_)
kmeans_label_Series

- 每个簇中的样本各有多少


In [ ]:
kmeans_label_Series.value_counts().sort_index()

- 簇质心的情况


In [ ]:
centers = pd.DataFrame(kmeans_row.cluster_centers_, columns=feature)
centers

- 对每个簇质心的值降序排列


In [ ]:
centers_t = centers.T
centers_t.columns = pd.Series(["cluster_0", "cluster_1"])
centers_t

In [ ]:
centers_t["cluster_0"].sort_values(ascending=False, inplace=False)

In [ ]:
centers_t["cluster_1"].sort_values(ascending=False, inplace=False)

## 实验结果分析:


1.最终确定的最优聚类簇数目及理由？


最终确定的最优聚类簇数目为 2 个  
理由如下：

- 根据轮廓系数(silhouette_score)评价 k-means 聚类模型，该值是簇密度与分散程度的评价指标，区间为[-1,1]之间，该值越接近 1，簇越紧凑，聚类越好
- 根据 CH 指标评价 k-means 聚类模型，值越大表示聚类效果越好
- 根据误差平方和评价 k-means 聚类模型，数值趋于稳定时达到最优

### 总结：综合以上三个系数，选择最优的聚类簇数目为 2


2.结合数据集分析两种聚类方法的优缺点和运用场景？


- K-Means
  - 优点：
    1. 原理简单，实现容易，收敛速度快
    2. 聚类效果较优
    3. 算法的可解释度比较强
    4. 主要需要调参的参数仅仅是簇数 k
  - 缺点：
    1. K 均值很难处理非球形的簇和不同大小的簇
    2. 对噪声、异常点比较敏感
    3. 如果各隐含类别的数据不平衡，比如各隐含类别的数据量严重失衡，或者各隐含类别的方差不同，则聚类效果不佳
    4. 采用迭代方法，得到的结果只是局部最优
    5. 初始质心的选取对聚类结果有很大影响
  - 运用场景：球形分布的簇，大小相近的簇
- DBSCAN
  - 优点：
    1. DBSCAN 能剔除噪声，对数据集中的异常点不敏感，自动确定簇个数
    2. 可以对任意形状的稠密数据集进行聚类
    3. 聚类结果没有偏倚
  - 缺点：
    1. 如果样本集的密度不均匀、聚类间距差相差很大时，聚类质量较差
    2. 如果样本集较大时，聚类收敛时间较长
    3. 调参稍复杂，主要需要对距离阈值 ϵ，邻域样本数阈值 MinPts 联合调参，不同的参数组合对最后的聚类效果有较大影响
  - 运用场景：数据集密度较为均匀，聚类间距相差不大


3.结合划分簇的质心取值，从医学角度分析聚类结果意义？


共 10 个指标：年龄、性别、体重指数、平均血压和 6 个血清测量值，从聚类结果来看，这 442 例患者的测量指标主要可以分为 2 类

- 一类的质心取值为 0.016759 0.015143 0.023623 0.020559 0.024194 0.025881 -0.024767 0.033360 0.029984 0.024538，表明年龄在这 442 例患者中在平均年龄再往上百分之 1.6 的患者，性别偏男性（或女性）的多百分之 1.5 的患者等等指标聚为一类
- 另一类的质心取值为 -0.017855 -0.016134 -0.025168 -0.021904 -0.025777 -0.027574 0.026387 -0.035543 -0.031945 -0.026143，明年龄在这这 442 例患者中在平均年龄再往下百分之 1.7 的患者，性别偏女性（或男性）的少百分之 1.6 的患者等等指标聚为一类


## 实验环境：

Anaconda Jupyter notebook、sklearn、matplotlib


## 教师评语:
